In [ ]:
import tweepy
import csv
import time
import datetime

consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True) #params will delay pull if rate exceeded

def pull_tweets(keywords, filename, day_since, day_until, stoppage=""):
    print("{} Started".format(datetime.datetime.now().strftime("%H:%M")))
    
    keywords = " OR ".join(keywords) #transform input list of keywords to required format
    print("Keywords selected: {}".format(keywords))
    search_query = keywords
    params_api = {
        "method":api.search,
        "q":search_query,
        "count":100, #max for standard licensing
        "lang":"en",
        "since": day_since, #if kernel dies or api gets banned can use "max_id" param instead to resume where it was let off
        "until": day_until,
        "result_Type":"recent", #pull tweets from most recent from the day specified in until param
        "include_entities":True
    }
    messages = []
    tweets_count = 0

    with open(filename, 'a') as csv_file:
        for tweet in tweepy.Cursor(**params_api).items():
            tweets_count += 1
            if str(tweet.created_at) == stoppage: #breaks the loop at a given time since we can not specify hours
                break
            if tweet.id_str not in messages:
                messages.append(tweet.id_str)
                print("Tweets recorded: {}. Last tweet date: {}.".format(tweets_count,tweet.created_at))
                csv.writer(csv_file).writerow([tweet.id_str, tweet.created_at, tweet.user.location, tweet.text])
                time.sleep(0.023) #if pulling tweets too fast API gets banned, using built-in delay and wait_on_rate_limit param
    print("{} Finished".format(datetime.datetime.now().strftime("%H:%M")))

In [ ]:
matches = {
    "FRAURU": [["#URUFRA", "#FRAURU"], "tweets-FRA-vs-URU.csv", "2018-07-06", "2018-07-07", "2018-07-06 13:59:59"],
    "BELBRA": [["#BELBRA", "#BRABEL"], "tweets-BEL-vs-BRA.csv", "2018-07-06", "2018-07-07", "2018-07-06 17:59:59"],
    "ENGSWE": [["#ENGSWE", "#SWEENG"], "tweets-ENG-vs-SWE.csv", "2018-07-07", "2018-07-08", "2018-07-07 13:59:59"],
    "CRORUS": [["#CRORUS", "#RUSCRO"], "tweets-CRO-vs-RUS.csv", "2018-07-07", "2018-07-08", "2018-07-07 17:59:59"],
    "FRABEL": [["#FRABEL", "#BELFRA"], "tweets-FRA-vs-BEL.csv", "2018-07-10", "2018-07-11", "2018-07-10 17:59:59"],
    "ENGCRO": [["#CROENG", "#ENGCRO"], "tweets-ENG-vs-CRO.csv", "2018-07-11", "2018-07-12", "2018-07-11 17:59:59"]
    }

pull_tweets(["#ENGSWE", "#SWEENG"], "tweets-ENG-vs-SWE.csv", "2018-07-07", "2018-07-08", stoppage="2018-07-07 13:59:59")